In [0]:
use redox.himss25;

with eg as (
  SELECT
    "post" as http_method
    ,"DiagnosticReport" as resource
    ,"_search" as action
    ,"subject=Patient/81c2f5eb-f99f-40c4-b504-59483e6148d7" as data
)
SELECT
  redox_request(
    http_method
    ,resource
    ,action
    ,data
  )
FROM 
  eg
;